In [1]:
!git clone https://github.com/HakureiPOI/Douban_Scraper.git

Cloning into 'Douban_Scraper'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (14/14), 107.93 KiB | 2.70 MiB/s, done.


In [2]:
import openpyxl
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

In [3]:
import pandas as pd
import requests
import json
import time
import re
import random
import logging
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [4]:
def setup_logger(name=__name__, log_file='/kaggle/working/Douban_Scraper/log.txt', level=logging.DEBUG):
    logger = logging.getLogger(name)
    logger.setLevel(level)

    formatter = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s')

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    return logger

In [5]:
logger = setup_logger()

In [6]:
class Interface():
    def __init__(self):
        self.session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(pool_connections = 100, pool_maxsize = 100)
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)
        self.user_agent = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        ]

    def _post(self, url, data, retries = 3, headers = None):
        for i in range(retries):
            try:
                headers = {
                    'User-Agent' : random.choice(self.user_agent)
                }

                response = self.session.post(url, data = data, headers = headers)
                response.raise_for_status()
                time.sleep(random.random() * 10)
                return response

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 403:
                    logger.warning(f'requests get error: {type(e).__name__}-{e}, skipping')
                    return None
                else:
                    logger.warning(f'requests get error on attempt {i + 1}, {type(e).__name__}-{e}')
                    time.sleep(3)

            except Exception as e:
                logger.warning(f'requests get error on attempt {i + 1}, {type(e).__name__}-{e}')
                time.sleep(3)

    def _get(self, url, retries = 3, headers = None):
        for i in range(retries):
            try:
                headers = {
                    'User-Agent' : random.choice(self.user_agent)
                }

                response = self.session.get(url, headers = headers)
                response.raise_for_status()
                time.sleep(random.random() * 10)
                return response

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 403:
                    logger.warning(f'requests get error: {type(e).__name__}-{e}, skipping')
                    return None
                else:
                    logger.warning(f'requests get error on attempt {i + 1}, {type(e).__name__}-{e}')
                    time.sleep(3)

            except Exception as e:
                logger.warning(f'requests get error on attempt {i + 1}, {type(e).__name__}-{e}')
                time.sleep(3)

In [7]:
api = Interface()

In [8]:
class Comment():
    def __init__(self, commentator, rating, content, datetime, form = 'short'):
        self.commentator = commentator
        self.rating = rating
        self.content = content
        self.datetime = datetime
        self.form = form

    def __str__(self):
        return f'{self.commentator}-{self.datetime}'

In [9]:
class Anima():
    def __init__(self, index, api):
        self.index = index
        self.url = f'https://movie.douban.com/subject/{index}/'
        self.api = api

        self.title = None
        self.shorts = []
        self.reviews = []

        self.dataframe = pd.DataFrame(columns = ['animal', 'commentator', 'datetime', 'rating', 'content'])

    def __str__(self):
        return f'{self.index}-{self.title}'

    def init(self, retries = 3):
        try:
            for i in range(retries):
                response = self.api._get(self.url)

                if not response:
                    logger.info(f'{self} init failed')
                    return False

                soup = BeautifulSoup(response.text, 'html.parser')
                title = soup.find('span', {'property' : 'v:itemreviewed'})

                if not title:
                    logger.info(f'{self} init waiting, retries : {i}')
                    time.sleep(random.randint(15, 30))
                    continue

                self.title = title.text
                logger.info(f'{self} init success')
                return True

            logger.warning(f'{self} init failed')
            return False

        except Exception as e:
            logger.warning(f'{self} init error, {type(e).__name__}-{e}')

    def __get_reviews(self, retries = 3):
        try:
            while True:
                logger.info(f'{self} get reviews, start : {len(self.reviews)}')

                data = {
                    'start': len(self.reviews),
                }
                url = f'https://movie.douban.com/subject/{self.index}/reviews'

                for i in range(retries):
                    response = self.api._post(url, data)

                    if not response:
                        logger.info(f'{self} get reviews success')
                        return True

                    soup = BeautifulSoup(response.text, 'html.parser')
                    reviews = soup.find_all('div', {'class' : 'main review-item'})

                    if not reviews:
                        logger.info(f'{self} get reviews waiting, retries : {i}')
                        time.sleep(random.randint(15, 30))
                        continue

                    for review in reviews:
                        commnet_id = review.attrs['id']
                        commentator = review.find('a', {'class' : 'name'}).text
                        rating = review.find('span', {'class' : ['allstar50', 'main-title-rating']})

                        if rating:
                            rating = rating.attrs['title']
                        else:
                            rating = None

                        datetime = review.find('span', {'class' : 'main-meta'}).text
                        full_comment = self.api._get(f'https://movie.douban.com/j/review/{commnet_id}/full')

                        if full_comment and full_comment.status_code == 200:
                            try:
                                content = json.loads(full_comment.text)['html']
                                content = BeautifulSoup(content, 'html.parser').text
                                self.reviews.append(Comment(commentator, rating, content, datetime, 'review'))
                            except json.JSONDecodeError as e:
                                logger.warning(f"Get comments error: Invalid JSON response - {e}")
                                break
                        else:
                            logger.warning(f"Get comments error: Request failed or empty response")

                    break

                if not reviews:
                    return False

        except Exception as e:
            logger.warning(f'{self} get comments error, {type(e).__name__}-{e}')

    def __get_shorts(self, retries = 3):
        try:
            while True:
                logger.info(f'{self} get shorts, start : {len(self.shorts)}')

                data = {
                    'start': len(self.shorts),
                }
                url = f'https://movie.douban.com/subject/{self.index}/comments'

                for i in range(retries):
                    response = self.api._post(url, data)

                    if not response:
                        logger.info(f'{self} get shorts success')
                        return True

                    soup = BeautifulSoup(response.text, 'html.parser')
                    comments = soup.find_all('div', {'class' : 'comment'})

                    if not comments:
                        logger.info(f'{self} get shorts waiting, retries : {i}')
                        time.sleep(random.randint(15, 30))
                        continue

                    for comment in comments:
                        comment_info = comment.find('span', {'class' : 'comment-info'})
                        commentator = comment_info.find('a').text
                        rating = comment_info.find('span', {'class' : ['allstar50', 'rating']})
                        if rating:
                            rating = rating.attrs['title']
                        else:
                            rating = None
                        datetime = comment_info.find('span', {'class' : 'comment-time'}).attrs['title']
                        content = comment.find('span', {'class' : 'short'}).text

                        self.shorts.append(Comment(commentator, rating, content, datetime, 'short'))

                    break

                if not comments:
                    return False

        except Exception as e:
            logger.warning(f'{self} get comments error, {type(e).__name__}-{e}')

    def get_comments(self):
        if not self.title:
            logger.warning(f'{self} get comments failed, init failed')
            return False

        self.__get_reviews()
        self.__get_shorts()

        for comment in self.reviews + self.shorts:
            self.dataframe = pd.concat([self.dataframe, pd.DataFrame([[self.title, comment.commentator, comment.datetime, comment.rating, comment.content]],
                                                                    columns = ['animal', 'commentator', 'datetime', 'rating', 'content'])])

        logger.info(f'{self} get comments success')
        return True

In [10]:
anima_df = pd.read_csv('/kaggle/working/Douban_Scraper/animas.csv')

In [11]:
animas = []

In [12]:
for index, row in anima_df.head(300).iterrows():
    animas.append(Anima(row['index'], api))

In [13]:
with ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(lambda anima : anima.init(), animas)

2024-09-21 02:59:02,559 - [INFO] - 26653375-我的英雄学院 僕のヒーローアカデミア init success
2024-09-21 02:59:02,800 - [INFO] - 36093351-葬送的芙莉莲 葬送のフリーレン init success
2024-09-21 02:59:03,375 - [INFO] - 36441528-青春猪头少年不会梦到红书包女孩 青春ブタ野郎はランドセルガールの夢を見ない init success
2024-09-21 02:59:03,862 - [INFO] - 10527275-来自新世界 新世界より init success
2024-09-21 02:59:04,958 - [INFO] - 4925398-命运石之门 STEINS;GATE init success
2024-09-21 02:59:05,986 - [INFO] - 36638192-败北女角太多了！ 負けヒロインが多すぎる！ init success
2024-09-21 02:59:05,988 - [INFO] - 36779574-仙逆 年番 init success
2024-09-21 02:59:06,560 - [INFO] - 36440342-我心里危险的东西 第二季 僕の心のヤバイやつ 2期 init success
2024-09-21 02:59:06,916 - [INFO] - 25851655-四月是你的谎言 四月は君の嘘 init success
2024-09-21 02:59:07,559 - [INFO] - 35801594-迷宫饭 ダンジョン飯 init success
2024-09-21 02:59:08,316 - [INFO] - 36765646-蓦然回首 ルックバック init success
2024-09-21 02:59:09,218 - [INFO] - 34456027-异度侵入 イド：インヴェイデッド ID:INVADED init success
2024-09-21 02:59:09,933 - [INFO] - 25850705-月刊少女野崎君 月刊少女野崎くん init success
2024-09-21 02:59:10,

In [14]:
with ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(lambda anima : anima.get_comments(), animas)

2024-09-21 03:05:23,242 - [INFO] - 34456027-异度侵入 イド：インヴェイデッド ID:INVADED get reviews, start : 0
2024-09-21 03:05:23,244 - [INFO] - 36440342-我心里危险的东西 第二季 僕の心のヤバイやつ 2期 get reviews, start : 0
2024-09-21 03:05:23,244 - [INFO] - 36779574-仙逆 年番 get reviews, start : 0
2024-09-21 03:05:23,245 - [INFO] - 35801594-迷宫饭 ダンジョン飯 get reviews, start : 0
2024-09-21 03:05:23,246 - [INFO] - 36765646-蓦然回首 ルックバック get reviews, start : 0
2024-09-21 03:05:23,248 - [INFO] - 36093351-葬送的芙莉莲 葬送のフリーレン get reviews, start : 0
2024-09-21 03:05:23,252 - [INFO] - 10527275-来自新世界 新世界より get reviews, start : 0
2024-09-21 03:05:23,253 - [INFO] - 36441528-青春猪头少年不会梦到红书包女孩 青春ブタ野郎はランドセルガールの夢を見ない get reviews, start : 0
2024-09-21 03:05:23,256 - [INFO] - 25851655-四月是你的谎言 四月は君の嘘 get reviews, start : 0
2024-09-21 03:05:23,265 - [INFO] - 26653375-我的英雄学院 僕のヒーローアカデミア get reviews, start : 0
2024-09-21 03:05:25,737 - [WARNING] - requests get error: HTTPError-403 Client Error: Forbidden for url: https://sec.douban.com/b?r=https%3A%2F%2Fm

In [15]:
comments = pd.concat([anima.dataframe for anima in animas])

In [16]:
comments

,animal,commentator,datetime,rating,content
0,异度侵入 イド：インヴェイデッド ID:INVADED,丘贝贝Kyubei,2020-03-19 21:04:19,还行,写作存在其共性：动画剧本也好，期刊论文与专著也罢，作者一定要有一个认知：不要想着在有限的空间...
0,异度侵入 イド：インヴェイデッド ID:INVADED,晓宇,2020-02-10 21:31:36,力荐,【写在最前边】感谢大家一直以来的点赞、回复、收藏、转发，一路走来大家辛苦了。13话的处理我是...
0,异度侵入 イド：インヴェイデッド ID:INVADED,zx2337,2020-01-27 11:58:34,力荐,把我写在B站的长评也搬过来一份吧看完第5集后，我在评论区发了关于JW真实身份的猜想，讨论了J...
0,异度侵入 イド：インヴェイデッド ID:INVADED,未来事务管理局,2020-04-08 22:17:59,力荐,“一部动画，能够在每集20分钟的螺蛳壳里做出如此多变的领域致敬，可以说是狂热粉丝和匠心制作...
0,异度侵入 イド：インヴェイデッド ID:INVADED,尤莲,2020-03-06 18:33:30,力荐,这两天被强烈安利了《异度侵入ID：INVADED》，男主史称话疗圣手。因而抽空看了。为了让自...
...,...,...,...,...,...
0,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,岁岁年年花相似,2023-02-04 22:14:02,推荐,闹腾 离谱 欢乐
0,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,热心市民两块钱,2018-10-07 21:47:56,推荐,这是我的童年没错了 现在看起来海绵宝宝其实很有深度 讽刺了一些社会现象
0,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,不高興,2021-08-23 22:44:35,推荐,前面有點社會寓言，後面還是蠻好看的，越來越理解到痞老闆很可愛，比基尼海灘永遠不可或缺的反面角...
0,海绵宝宝 The SpongeBob Movie: Sponge Out of Water,十六太,2022-07-11 19:16:24,推荐,补


In [17]:
def remove_illegal_chars(value):
    if isinstance(value, str):
        return ILLEGAL_CHARACTERS_RE.sub(r'', value)
    else:
        return value

In [18]:
comments = comments.map(remove_illegal_chars)

In [19]:
comments.to_excel('/kaggle/working/Douban_Scraper/comments.xlsx', index = False)